In [1]:
!gdown --id 1baHMGphE705ikbV8hCk5_3FtanZ_dpSt
!gdown --id 1RR1LkayEgPTJql5-ITQmtsoTom9NRoOH

Downloading...
From: https://drive.google.com/uc?id=1baHMGphE705ikbV8hCk5_3FtanZ_dpSt
To: /content/aligned.tar.gz
2.81GB [00:41, 68.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RR1LkayEgPTJql5-ITQmtsoTom9NRoOH
To: /content/train_val_txt_files_per_fold.zip
3.11MB [00:00, 101MB/s]


In [2]:
!unzip -n -q train_val_txt_files_per_fold.zip
!tar -xf aligned.tar.gz

In [ ]:
!pip install pytorch-lightning
!pip install comet-ml
!pip install local-attention
!pip install einops

In [1]:
from hparams import hyperparams
hyperparams.path_to_files

'/content'

In [2]:
from depen import *
from model import Age_Gender_pl
from datasets import Adience_pl_dataset

from comet_ml import Experiment
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [3]:
class CheckpointEveryNSteps(pl.Callback):
    """
    Save a checkpoint every N steps, instead of Lightning's default that checkpoints
    based on validation loss.
    """

    def __init__(
        self,
        save_step_frequency,
        prefix="N-Step-Checkpoint",
        use_modelcheckpoint_filename=False,
    ):
        """
        Args:
            save_step_frequency: how often to save in steps
            prefix: add a prefix to the name, only used if
                use_modelcheckpoint_filename=False
            use_modelcheckpoint_filename: just use the ModelCheckpoint callback's
                default filename, don't use ours.
        """
        self.save_step_frequency = save_step_frequency
        self.prefix = prefix
        self.use_modelcheckpoint_filename = use_modelcheckpoint_filename

    def on_batch_end(self, trainer: pl.Trainer, _):
        """ Check if we should save a checkpoint after every train batch """
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            if self.use_modelcheckpoint_filename:
                filename = trainer.checkpoint_callback.filename
            else:
                filename = f"{self.prefix}_{epoch}_{global_step}.ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)

In [4]:
seed_e(42)

In [5]:
comet_logger = CometLogger(
    save_dir='/content/log/',
    api_key="23CU99n7TeyZdPeegNDlQ5aHf",
    project_name="dl-proj",
    workspace="etzelkut",
    # rest_api_key=os.environ["COMET_REST_KEY"], # Optional
    # experiment_name="default" # Optional
)

CometLogger will be initialized in online mode


In [6]:
dataset_pl = Adience_pl_dataset(hyperparams)
dataset_pl.prepare_data()
dataset_pl.setup()
train_loader = dataset_pl.train_dataloader()
steps_per_epoch = int(len(train_loader))
print(steps_per_epoch)

already prepared but later add
ready
396


In [7]:
checkpoint_callback = ModelCheckpoint(
                                      monitor='val_loss',
                                      save_last=True, 
                                      dirpath= "/content/drive/MyDrive/dl_proj/gg/",#os.path.join(path, "/checkpoints"),
                                      filename='samplemodel{epoch}.ckpt',
                                      save_top_k=3,
                                      mode='min',
                                      )
every_epoch = CheckpointEveryNSteps(save_step_frequency = steps_per_epoch, use_modelcheckpoint_filename = True)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Checkpoint directory /content/drive/MyDrive/dl_proj/gg/ exists and is not empty. With save_top_k=3, all files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


In [8]:
model = Age_Gender_pl(hyperparams, steps_per_epoch = steps_per_epoch)

softmax
glu


In [9]:
"""for batch in train_loader:
  images, labels = batch['image'], batch['label']
  break
!cp "/content/drive/MyDrive/dl_proj/improved-net.pt" "/content"
PATH = "/content/improved-net.pt"
model.network.load_state_dict(torch.load(PATH))"""
#model.network = model.network.train()

'for batch in train_loader:\n  images, labels = batch[\'image\'], batch[\'label\']\n  break\n!cp "/content/drive/MyDrive/dl_proj/improved-net.pt" "/content"\nPATH = "/content/improved-net.pt"\nmodel.network.load_state_dict(torch.load(PATH))'

In [8]:
trainer = Trainer(callbacks=[checkpoint_callback, every_epoch], #lr_monitor],
                    logger=comet_logger,
                    gpus=1,
                    profiler=True,
                    #auto_lr_find=True, #set hparams
                    #gradient_clip_val=0.5,
                    check_val_every_n_epoch=2,
                    #early_stop_callback=True,
                    max_epochs = hyperparams.epochs,
                    #min_epochs=400,
                    progress_bar_refresh_rate = 0,
                    deterministic=True,)

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [21]:
trainer.fit(model, dataset_pl)


  | Name    | Type             | Params
---------------------------------------------
0 | network | Model_dl         | 65.1 M
1 | loss    | CrossEntropyLoss | 0     


RuntimeError: ignored

In [13]:
trainer.test()

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/etzelkut/dl-proj/e3cb9e446e634a69ac565b423c111a2d



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'epoch_now': 33,
 'test_acc': tensor(43.5460, device='cuda:0'),
 'test_loss': tensor(1.8741, device='cuda:0'),
 'train_acc': tensor(83.5006, device='cuda:0'),
 'train_loss': tensor(0.4421, device='cuda:0'),
 'train_loss_epoch': tensor(0.4695, device='cuda:0'),
 'train_loss_step': tensor(0.4421, device='cuda:0'),
 'val_loss': tensor(1.0260, device='cuda:0'),
 'vall_acc': tensor(67.7397, device='cuda:0')}
--------------------------------------------------------------------------------


[{'epoch_now': 33,
  'test_acc': 43.54597091674805,
  'test_loss': 1.8741251230239868,
  'train_acc': 83.50056457519531,
  'train_loss': 0.44212615489959717,
  'train_loss_epoch': 0.4694541394710541,
  'train_loss_step': 0.44212615489959717,
  'val_loss': 1.0259755849838257,
  'vall_acc': 67.73967742919922}]

In [12]:
checkpoint_name = os.path.join("/content/drive/MyDrive/dl_proj/gg", 'manual_save' + str(228) + '.ckpt')
trainer.save_checkpoint(checkpoint_name)

In [ ]:
#########################################################################################

In [19]:
!cp "/content/drive/MyDrive/dl_proj/gg/last.ckpt" "/content"
PATH = "/content/last.ckpt"

In [20]:
model = Age_Gender_pl.load_from_checkpoint(PATH)
trainer.test(model, dataset_pl.test_dataloader())

softmax
glu
--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(39.6526, device='cuda:0'),
 'test_loss': tensor(2.1840, device='cuda:0')}
--------------------------------------------------------------------------------


[{'test_acc': 39.652591705322266, 'test_loss': 2.184048891067505}]

In [16]:
from comet_ml import Experiment
from comet_ml import ConfusionMatrix

In [17]:
def train_index_to_example(index):
  tmp, _ = train_dataset[index]
  img = tmp.numpy()[0]
  data = comet_logger.experiment.log_image(img, name="train_%d.png" % index)
  return {"sample": str(index), "assetId": data["imageId"]}

def onehot(i):
  v = [0] * 8
  v[i] = 1
  return v

confusion_matrix = ConfusionMatrix(index_to_example_function=train_index_to_example) 

In [11]:
test_loader = dataset_pl.test_dataloader()

In [18]:
images = []
labels_true = []
predictions = []

for batch in test_loader:
  images, labels = batch['image'], batch['label']
  labels = labels.tolist() #.flatten()
  labels_true.extend(labels)
  pred = model(images.to(device))
  v, i = pred.cpu().topk(1)
  i = i.flatten().tolist()
  predictions.extend(i)

In [ ]:
labels_true

In [ ]:
onehot_labels = [onehot(label) for label in labels_true]
onehot_preds = [onehot(pred) for pred in predictions]

In [ ]:
confusion_matrix.compute_matrix(onehot_labels, onehot_preds)

In [ ]:
comet_logger.experiment.log_confusion_matrix(
            matrix=confusion_matrix,
            title="Train Confusion Matrix",
            file_name="train-confusion-matrix.json")

{'api': 'https://www.comet.ml/api/rest/v2/experiment/asset/get-asset?assetId=2f7e3291812942c98f5bd2d0f5fa969e&experimentKey=b39a887e10834a4bab38f8897ac4e58f',
 'assetId': '2f7e3291812942c98f5bd2d0f5fa969e',
 'web': 'https://www.comet.ml/api/asset/download?assetId=2f7e3291812942c98f5bd2d0f5fa969e&experimentKey=b39a887e10834a4bab38f8897ac4e58f'}

In [ ]:
from pathlib import Path

In [ ]:
IMAGE_EXTS = ['.jpg', '.png', '.jpeg']
def expand_greyscale(t):
    return t.expand(3, -1, -1)

class ImagesDataset(Dataset):
    def __init__(self, folder, image_size):
        super().__init__()
        self.folder = folder
        self.paths = []

        for path in Path(f'{folder}').glob('**/*'):
            _, ext = os.path.splitext(path)
            if ext.lower() in IMAGE_EXTS:
                self.paths.append(path)

        print(f'{len(self.paths)} images found')

        self.transform = transforms.Compose([
            transforms.Resize(image_size),
            transforms.CenterCrop(image_size),
            transforms.ToTensor(),
            transforms.Lambda(expand_greyscale)
        ])

    def __len__(self):
        return len(self.paths)


    def __getitem__(self, index):
        path = self.paths[index]
        img = Image.open(path)
        img = img.convert('RGB')
        return self.transform(img)

In [ ]:
!cp "/content/drive/MyDrive/dl_proj/imim.zip" "/content"

In [ ]:
!unzip -n -q imim.zip

In [ ]:
photos = ImagesDataset("/content/pict", 256)

4 images found


In [ ]:
photos[i].shape

torch.Size([3, 256, 256])

In [ ]:
for i in range(len(photos)):
  p = photos[i].to(device)
  p = p.unsqueeze(0)
  pred = model(p)
  v, i = pred.cpu().topk(1)
  print(i)

tensor([[4]])
tensor([[5]])
tensor([[3]])
tensor([[7]])


In [13]:
import sklearn
from sklearn.metrics import confusion_matrix
a = confusion_matrix(labels_true, predictions, normalize = "true")

In [14]:
for row in a:
    for elem in row:
        print("%.2f"%(elem*100), end="\t")
    print("")

48.21	36.07	5.95	2.38	4.05	2.26	0.24	0.83	
15.82	37.00	26.27	8.31	6.70	4.56	0.27	1.07	
7.58	21.05	36.42	4.63	19.16	6.32	3.37	1.47	
0.00	1.09	10.22	5.84	58.03	17.88	2.55	4.38	
0.00	1.80	8.62	5.53	59.07	20.59	2.06	2.32	
0.72	0.36	11.91	2.89	40.79	31.77	3.97	7.58	
0.83	2.50	7.50	9.17	25.00	35.83	7.50	11.67	
1.97	1.48	8.37	0.49	16.75	33.99	7.39	29.56	


In [15]:
#copied from vgg
# one-off accuracy

conf_mat = a
ncls = len(conf_mat)
tot_acc = 0

for i in range(ncls):
    
    acc = conf_mat[i][i]

    # add left
    if i > 0:
        acc += conf_mat[i][i-1]
    
    if i < ncls - 1:
        acc += conf_mat[i][i+1]
    
    tot_acc += acc

tot_acc = tot_acc / ncls
print("%.2f" % (tot_acc * 100))

69.16


In [ ]:
#model = Age_Gender_pl(hyperparams, steps_per_epoch)
#!cp "/content/drive/MyDrive/dl_proj/gg/first_try_color/last.ckpt" "/content"
#PATH = "/content/last.ckpt"
#model = Age_Gender_pl.load_from_checkpoint(PATH, hparams = hyperparams)
#lr_monitor = LearningRateMonitor(logging_interval='step')
#trainer.test(model, dataset_pl.test_dataloader())